In [14]:
import torch
import customDataset
import os
import lfcm
from dotenv import load_dotenv
from torch.utils.data import DataLoader

load_dotenv()

True

In [15]:
root_dir = os.getenv('ROOT_PATH')
split_test = 'test.csv'
batch_size = 24
workers = 4
fcm_model_name = "fcm_v2_b24.pth"
lfcm_model_name = "lfcm_v2_b24.pth"
gpus = [0]
gpu = 0
CUDA_VISIBLE_DEVICES = 0

In [16]:
fcm_result_fpath = f"{root_dir}/results/fcm_result.txt"
lfcm_result_fpath = f"{root_dir}/results/lfcm_result.txt"


In [17]:
fcm_state_dict = torch.load(f"{root_dir}/checkpoints/{fcm_model_name}", map_location={'cuda:1':'cuda:0', 'cuda:2':'cuda:0', 'cuda:3':'cuda:0'})
lfcm_state_dict = torch.load(f"{root_dir}/checkpoints/{lfcm_model_name}", map_location={'cuda:1':'cuda:0', 'cuda:2':'cuda:0', 'cuda:3':'cuda:0'})


In [18]:
# print(fcm_state_dict)
# print(lfcm_state_dict)

In [19]:
fcm_model = lfcm.OldModel()
lfcm_model = lfcm.LFCM()

Inception v3 pretraining: Hardcoded strict=False, Might be omiting layers
Inception v3 pretraining: Hardcoded strict=False, Might be omiting layers


In [20]:
fcm_model = torch.nn.DataParallel(fcm_model, device_ids=gpus).cuda(gpu)

# print(fcm_model)

In [21]:
lfcm_model = torch.nn.DataParallel(lfcm_model, device_ids=gpus).cuda(gpu)

# print(lfcm_model)

In [22]:
fcm_model.load_state_dict(fcm_state_dict)
lfcm_model.load_state_dict(lfcm_state_dict)

<All keys matched successfully>

In [23]:
fcm_test_dataset = customDataset.CustomDatasetFCM(root_dir, split_test, Rescale=299, RandomCrop=0, Mirror=False, embed_dir='test_embeddings')

fcm_test_loader = DataLoader(
    fcm_test_dataset, batch_size=batch_size, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=None)


In [24]:
lfcm_test_dataset = customDataset.CustomDatasetLFCM(root_dir, split_test, Rescale=299, RandomCrop=0, Mirror=False, embed_dir='test_embeddings')

lfcm_test_loader = DataLoader(
    lfcm_test_dataset, batch_size=batch_size, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=None)

In [26]:
# fcm_result_fpath = f"{root_dir}/results/fcm_result.txt"
fcm_out_file = open(fcm_result_fpath, "w")

# FCM 
with torch.no_grad():
    fcm_model.eval()
    for i,(image, image_text, tweet, target) in enumerate(fcm_test_loader):
        image_var = torch.autograd.Variable(image)
        image_text_var = torch.autograd.Variable(image_text)
        tweet_var = torch.autograd.Variable(tweet)

        outputs = fcm_model(image_var, image_text_var, tweet_var)

        for idx,el in enumerate(outputs):
            topic_probs_str = ''
            for t in el:
                topic_probs_str = topic_probs_str + ',' + str(float(t))
            fcm_out_file.write(str(int(target[idx])) + topic_probs_str + '\n')

fcm_out_file.close()

In [27]:
# fcm_result_fpath = f"{root_dir}/results/fcm_result.txt"
lfcm_out_file = open(lfcm_result_fpath, "w")

# FCM 
with torch.no_grad():
    lfcm_model.eval()
    for i,(image, image_text, tweet, comment, target) in enumerate(lfcm_test_loader):
        image_var = torch.autograd.Variable(image)
        image_text_var = torch.autograd.Variable(image_text)
        tweet_var = torch.autograd.Variable(tweet)
        comment_var = torch.autograd.Variable(comment)

        outputs = lfcm_model(image_var, image_text_var, tweet_var, comment_var)

        for idx,el in enumerate(outputs):
            topic_probs_str = ''
            for t in el:
                topic_probs_str = topic_probs_str + ',' + str(float(t))
            lfcm_out_file.write(str(int(target[idx])) + topic_probs_str + '\n')

lfcm_out_file.close()

In [ ]:
# evaluate_model.run_evaluation('fcm_e16b16','classification')

# evaluate_model.run_evaluation('lfcm_e16b16','classification')